In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

# Board Dimensions
number_of_squares_X = 10
number_of_squares_Y = 7
nX = number_of_squares_X - 1
nY = number_of_squares_Y - 1
square_size = 0.021

# Arrays to store objects and images
object_points = []
image_points = []

# Termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 3D Points on real space
object_points_3D = np.zeros((nX * nY, 3), np.float32)
object_points_3D[:, :2] = np.mgrid[0:nX, 0:nY].T.reshape(-1, 2) * square_size

# Folder where the images are storage
notebook_dir = os.getcwd()
project_dir = os.path.dirname(notebook_dir)
image_folder = os.path.join(project_dir, 'img', '1.calibracionCamara')
image_files = glob.glob(os.path.join(image_folder, '*.jpg'))

def CalibrateCamera():
    if not image_files:
        print("No se encontraron imagenes.", image_folder)
        return

    # Calibration using chess board images, Finds chessboard corners
    for image_file in image_files:
        image = cv2.imread(image_file)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        success, corners = cv2.findChessboardCorners(gray, (nX, nY), None)

        #Find corners on the images
        if success:
            object_points.append(object_points_3D)
            corners_2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            image_points.append(corners_2)
            cv2.drawChessboardCorners(image, (nX, nY), corners_2, success)

    if len(image_points) == 0:
        print("No corners detected in any image. Check your chessboard patterns.")
        return

    # Camera Calibration Data
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)

    print("Camera matrix:\n", mtx)
    print("Distortion coefficients:\n", dist)
    print("Distortion coefficients [k1, k2, p1, p2, k3]:\n", dist.ravel())
    print("Reprojection error (RMS):", ret)

    height, width = image.shape[:2]
    print(f"Altura: {height}")
    print(f"Ancho: {width}")
    center = (width // 2, height // 2)

    print("Coordenadas centro ", center)

    # Undistort all the images
    for image_file in image_files:
        image = cv2.imread(image_file)
        if image is None:
            print(f"Failed to load image: {image_file}")
            continue

        height, width = image.shape[:2]
        new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))
        undistorted = cv2.undistort(image, mtx, dist, None, new_camera_mtx)

        #View the Images
        plt.imshow(image)
        plt.title("Visualizacion de la imagen Original: " + os.path.basename(image_file))
        plt.axis('off')
        #plt.show()

        plt.imshow(undistorted)
        plt.title("Visualizacion de la imagen Corregida: " + os.path.basename(image_file))
        plt.axis('off')
        plt.show()

        # Save all the images on a new folder
        base_filename = os.path.basename(image_file)
        name, ext = os.path.splitext(base_filename)
        output_path = os.path.join(os.path.join(project_dir, 'results', '1.calibracionCamara'), name + '_undistorted' + ext)
        cv2.imwrite(output_path, undistorted)

    # Create a text file with the camera Data
        # Define output folder
        output_folder = os.path.join(project_dir, 'results', '1.calibracionCamara')
        file_path = os.path.join(output_folder, 'parametros.txt')

        # Write camera calibration data to the file
        with open(file_path, "w") as f:
           f.write("Optimal Camera matrix:")
           f.write(np.array_str(new_camera_mtx))

           f.write("\n Coeficiente de distorsion:")
           f.write(np.array_str(dist))

           f.write("\n Vectores de rotacion:")
           f.write(', '.join(str(rvec) for rvec in rvecs))

           f.write("\n Vectores de tralacion:")
           f.write(', '.join(str(tvec) for tvec in tvecs))

           f.write("Error RMS de reproyección: " + str(ret) + "\n")
           # File close

    print("All images undistorted and saved.")

CalibrateCamera()

No se encontraron imagenes. /img/1.calibracionCamara


Punto 1 Calibracion de Camaras

1. Matriz de Camara K

K =  [[3.16386392e+03 0.00000000e+00 2.00104091e+03]
      [0.00000000e+00 3.18629093e+03 1.55433682e+03]
      [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

2. Coeficientes de distorsion son:

[ 0.38057019 -1.04938422 -0.00343441  0.00788838  0.07396761]

k1 = 0.38057019
k2 = -1.04938422
p1 = -0.00343441  
p2 = 0.00788838  
k3 = 0.07396761

3. Error RMS de reproyección

RMS: 0.9848851012377567

4. ¿Qué tipo de distorsión predomina? (barril, cojín)

R/= Se evidencia que en la mayoria de las imagenes predomina la forma de cojin por sobre la de barril, si nos centramos en las imagenes corregidas podremos ver que las correciones ocurren no en las esquinas de la imagen si no en los 4 lados en la posicion del centro, de cierta forma vemos como las imagenes se hunden ligeramente hacia el centro y predominantemente en en los lados horizontales de la imagen dando la impresion de que crean un cojin, aunque para este caso no estamos viendo un cojin completo ya en las esquinas la distorsion no es tan ponunciada.

5. ¿La longitud focal fx es similar a fy? ¿Por qué podría diferir?

fx = 3.16386392e+03 y fy = 3.18629093e+03

R/= Los puntos fx y fy, se encuentran cercanas entre si diferiendo principalmente en valores decimales, si bien es cierto que normalmente pueden ser iguales el focal lenght como se le nombra a estos dos puntos que representan las direcciones en x y en y podran diferir dependiendo de los pixeles de la camara.

6. ¿El punto principal (cx, cy) está cerca del centro de la imagen?

(cx, cy) = 2.00104091e+03 , 1.55433682e+03

(ix, iy) = 2016, 1512

Dado que 2.00104091e+03  =  2.00104091 x 10³
                         =  2001.04091
         1.55433682e+03  =  1.55433682 x 10³
                         =  1554.33682

Tenemos que tanto cx y cy que serian las coordenadas del centro optico que hemos sacado de acuerdo a la calibracion, estos dos puntos estan relativamente cerca del centro original aunque notamos una mayor variacion en los valores de y, lo cual podria deberse al echo de que son los lados superior e inferior los que presenta una mayor deformacion en las imagenes corregidas, en este caso estariamos hablando de que cx esta a 14 pixeles a la izquierda del centro de la imagen y que y el cual esta mas separado, esta 42 pixeles por encima del centro de la imagen.
        